In [1]:
# Definitions and functions
%matplotlib inline
from __future__ import division
import pdb
import sys
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib
import scipy.signal as sg
import math
import scipy as sp
import socket
import os
import wave
import struct
import h5py
from scipy.io import wavfile
from scipy.signal import spectrogram
import logging
from IPython.display import display
matplotlib.style.use('ggplot')
from ipywidgets import *
import yaml
import numpy.lib.recfunctions as rfn 
import numpy.matlib as npm
import glob
import tarfile as trf
from contextlib import closing 

# Check wich computer to decide where the things are mounted
comp_name=socket.gethostname()
print('Computer: ' + comp_name)

if 'txori' in comp_name or 'passaro' in comp_name or 'lintu' in comp_name:
    repos_folder = os.path.abspath('/mnt/cube/earneodo/repos')
    experiment_folder = os.path.join('/mnt/cube/earneodo/bci_zf/')

sys.path.append(os.path.join(repos_folder, 'ephysflow'))
sys.path.append(os.path.join(repos_folder, 'swissknife'))

# Careful with these: basic_viewing should be replaced soon
from basic_viewing import events as evt
# from basic_viewing.structure import kwik_functions as kwkf
# from basic_viewing.structure.core import h5_functions as kwdf

from bci.core import kwik_functions as kwkf
from bci.core.file import h5_functions as kwdf 
from bci.core import expstruct as et
from archivetools import tar as tt


logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)



logger.debug('All modules loaded')

Computer: passaro


2017-02-08 16:08:39,358 root         DEBUG    All modules loaded


In [2]:
sys.version_info[0] >= 3

True

In [15]:
# bird and session
bird = 'z020'
#sess = 'day-anesth_surface_2016-11-14_6'
#stim_sess = '001'

#exp_par = et.get_parameters(bird, sess)
#fn = et.file_names(bird, sess)

In [37]:
import importlib
importlib.reload(tt)
d_f, md_arch, md_list = tt.compress_folder(os.path.join(arch_fold, arch_sess), '/usr/local/experiment/tests')
check_list, _ = tt.check_tar_archive(d_f)
checked_list, check_ok = tt.check_tar_archive(d_f, md_checklist=check_list)

2017-01-29 01:59:38,505 archivetools.tar INFO     Will compress folder 2016-06-16_16-28-28_150 from /Data/bci_zf/raw_data/z017 into /usr/local/experiment/tests/2016-06-16_16-28-28_150.tar.
2017-01-29 01:59:38,512 archivetools.tar DEBUG    md5_wrapper: entered /Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150/experiment1.kwx
2017-01-29 01:59:38,513 archivetools.tar DEBUG    Adding /Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150/experiment1.kwx as experiment1.kwx
2017-01-29 01:59:38,664 archivetools.tar DEBUG    md5_wrapper: entered /Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150/experiment1.kwe
2017-01-29 01:59:38,665 archivetools.tar DEBUG    Adding /Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150/experiment1.kwe as experiment1.kwe
2017-01-29 01:59:38,667 archivetools.tar DEBUG    md5_wrapper: entered /Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150/settings.xml
2017-01-29 01:59:38,669 archivetools.tar DEBUG    Adding /Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150/settin

In [62]:
# A more efficient way, codewise, but not the way I like it
import shutil as sh
arname = sh.make_archive(os.path.join(arch_fold, arch_sess), 'xztar',
                         root_dir=os.path.join(arch_fold, arch_sess),
                         base_dir='./',
                         logger=logger)

2017-01-27 19:27:59,539 root         DEBUG    changing into '/Data/bci_zf/raw_data/z017/2016-06-16_16-28-28_150'
2017-01-27 19:27:59,541 root         INFO     Creating tar archive
2017-01-27 19:27:59,826 root         DEBUG    changing back to '/mnt/cube/earneodo/repos/ephysflow'


In [5]:
import shutil

import importlib

In [23]:
from bci.core.file import file_functions as ff
importlib.reload(ff)
fn_b = et.flex_file_names(bird, location='experiment')
l_cube = ff.list_folders(fn_b['folders']['raw'])

fn_store = et.flex_file_names(bird, location='store')
l_store = ff.list_folders(fn_store['folders']['raw'])

fn_store = et.flex_file_names(bird, location='experiment_local')
l_local=ff.list_folders(fn_store['folders']['raw'])

fn_archive = et.flex_file_names(bird, location='archive')
l_archived = ff.list_folders(fn_archive['folders']['raw'])

In [24]:
fn_archive

{'folders': {'prb': '/mnt/cube/earneodo/archive/bci_zf/probes',
  'raw': '/mnt/cube/earneodo/archive/bci_zf/raw_data/z020/',
  'rw': '/mnt/cube/earneodo/archive/bci_zf/raw_data/z020/',
  'ss': '/mnt/cube/earneodo/archive/bci_zf/ss_data/z020/',
  'stim': '/mnt/cube/earneodo/archive/bci_zf/stim_data/z020/',
  'templ': '/mnt/cube/earneodo/archive/bci_zf/templates',
  'tmp': '/mnt/cube/earneodo/archive/bci_zf/tmp_data/z020/'},
 'locations': {'archive': '/mnt/cube/earneodo/archive/bci_zf',
  'experiment': '/mnt/cube/earneodo/bci_zf',
  'experiment_local': '/usr/local/experiment',
  'repos': '/mnt/cube/earneodo/repos',
  'scratch': '/usr/local/experiment/scratchpad',
  'store': '/Data/bci_zf'},
 'structure': {'cand': 'experiment.mot.h5',
  'evt': 'experiment.kwe',
  'kk_par': 'params.prm',
  'kk_prb': '*.prb',
  'ks_cfg': 'config.m',
  'ks_map': 'chanMap.mat',
  'ks_mas': 'master.m',
  'ks_par': 'params.py',
  'mic': 'experiment-rec_000.mic.wav',
  'par': 'experiment.par.yml',
  'sng': 'expe

In [18]:
l_cube

['/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-03_10-53-56_100',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/001',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-02_20-22-38_100',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-02_11-10-42_2100_00',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/sleep_remote_2016-06-05_00-54-57_100',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-06_10-47-28_150',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-08_15-28-10_150',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-05_07-51-42_100',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-03_16-12-39_100',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-15_14-05-24_150',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-06_09-15-56_150',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-27_16-33-20_225',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-05_11-18-10_150',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-02_19-16-11_2100',
 '/mnt/cube/earneodo/bci_zf/raw_data/z020/2016-06-

In [22]:
for i in l_local + l_store:
    if not os.path.split(i)[-1] in [os.path.split(l)[-1] for l in l_cube]:
        print(i)
        shutil.copytree(i, os.path.join(fn_b['folders']['raw'], os.path.split(i)[-1]))

/Data/bci_zf/raw_data/z020/2016-06-04_17-46-03_100


In [30]:
def archive_data(bird, sess, 
                 fold_type = 'raw', 
                 locations={'orig': 'store', 
                            'dest':'archive', 
                            'scratch':'scratch'}, 
                 mode='xz',
                 use_scratch=False):
    # mode options ''(no compression), 'xz','gz', 'bz2' refer to tarfile modes.
    # scratch is optional
    
    logger.info('Will archive {0} data from bird {1}, sess {2}'.format(fold_type, bird, sess))
    f_struct = {key : et.flex_file_names(bird, sess, location=item) for key, item in locations.items()}
    f_struct['dest'] = et.flex_file_names(bird, location=locations['dest'])
    
    source_folder = f_struct['orig']['folders'][fold_type]
    logger.info('Source folder: {}'.format(source_folder))
    
    
    if use_scratch:
        logger.debug('Using scratch')
        tmp_folder = f_struct['scratch']['folders']['tmp']
        dest_folder = f_struct['scratch']['folders'][fold_type]
        logger.info('Will copy source into tmp {0}\n and use as source_folder'.format(tmp_folder))
        logger.info('Cleaning up temp folder')
        shutil.rmtree(tmp_folder)
        shutil.copytree(source_folder,tmp_folder)
        source_folder = tmp_folder
        logger.debug('Source folder is now {0}\n'.format(source_folder))

        
    else:
        dest_folder = f_struct['dest']['folders'][fold_type]
    assert(not (source_folder == dest_folder))
        
    shutil.rmtree(dest_folder)
    et.mkdir_p(dest_folder)
    logger.info('Dest folder: {}'.format(dest_folder))
    d_f, md_arch, md_list = tt.compress_folder(source_folder, dest_folder, mode='w:{}'.format(mode))
    check_list, check_ok = tt.check_tar_archive(d_f, md_checklist=md_list)
    
    if not check_ok:
        raise RuntimeError('Archive {} md5 hashes dont match the originals')
    
    if use_scratch:
        logger.debug('Have to move from sctratch to dest')
        final_dest_folder = f_struct['dest']['folders'][fold_type]
        logger.info('Final dest folder: {}'.format(final_dest_folder))
        shutil.move(dest_folder, final_dest_folder)
        logger.debug('Cleaning up tmp folder {}'.format(tmp_folder))
        shutil.rmtree(tmp_folder)
    
    return f_struct, check_list

#fstru, chk = archive_data(bird, arch_sess, use_scratch=True)

In [32]:
fstru['scratch']['folders']['tmp']

'/usr/local/experiment/scratchpad/tmp_data/z017/2016-07-11_14-49-38_225'

In [34]:
fstru['orig']['folders'][data_folder]

'/mnt/cube/earneodo/bci_zf/raw_data/z017/2016-07-11_14-49-38_225'

In [30]:
data_folder = 'raw'
log_fn = et.file_names(bird)
log_file_path = os.path.join(log_fn['folders']['ss'], 
                             'archive_all_{}.log'.format(data_folder))
fh = logging.FileHandler(log_file_path)
formatter = logging.Formatter('%(asctime)s - %(threadName)-10s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)

In [ ]:
locations={'orig': 'experiment', 
        'dest':'archive', 
        'scratch':'scratch'}

fn_b = et.flex_file_names(bird, location=locations['orig'])
l_cube = ff.list_folders(fn_b['folders'][data_folder])
bird_archived = []

for s in l_cube:
    logger.info('Archiving {}'.format(s))
    sess = os.path.split(s)[-1]
    fstru, chk = archive_data(bird, sess, fold_type=data_folder, locations=locations, mode='xz', use_scratch=True)
    bird_archived.append([s, fstru, chk])

2017-02-01 10:22:58,422 root         INFO     Archiving /mnt/cube/earneodo/bci_zf/raw_data/z017/2016-07-11_14-49-38_225
2017-02-01 10:22:58,439 root         INFO     Will archive raw data from bird z017, sess 2016-07-11_14-49-38_225
2017-02-01 10:22:58,444 root         INFO     Source folder: /mnt/cube/earneodo/bci_zf/raw_data/z017/2016-07-11_14-49-38_225
2017-02-01 10:22:58,447 root         DEBUG    Using scratch
2017-02-01 10:22:58,451 root         INFO     Will copy source into /usr/local/experiment/scratchpad/tmp_data/z017/2016-07-11_14-49-38_225
 and use as source_folder
2017-02-01 10:23:00,266 root         DEBUG    Source folder is now /usr/local/experiment/scratchpad/tmp_data/z017/2016-07-11_14-49-38_225

2017-02-01 10:23:00,269 bci.core.expstruct DEBUG    Creating directory /usr/local/experiment/scratchpad/raw_data/z017/2016-07-11_14-49-38_225
2017-02-01 10:23:00,271 root         INFO     Dest folder: /usr/local/experiment/scratchpad/raw_data/z017/2016-07-11_14-49-38_225
2017-0

In [29]:
locations={'orig': 'experiment', 
        'dest':'archive', 
        'scratch':'scratch'}

def archive_bird(bird, locations = {'orig': 'experiment',
                                    'dest':'archive',
                                    'scratch':'scratch'},
                 data_folder='raw', 
                 overwrite=False):
    
    fn_orig = et.flex_file_names(bird, location=locations['orig'])
    fn_dest = et.flex_file_names(bird, location=locations['dest'])
    
    orig_folders_list = ff.list_folders(fn_orig['folders'][data_folder])
    
    dest_folder_names_list = [os.path.split(s)[-1] for s in ff.list_folders(fn_dest['folders'][data_folder])]
    
    bird_archived = []
    
    for s in orig_folders_list:
        sess = os.path.split(s)[-1]
        if sess in dest_folder_names_list:
            logger.info('{0} already present in {1}'.format(sess, fn_dest['folders'][data_folder]))
            if overwrite:
                logger.warning('Will delete {} to overwrite'.format(os.path.join(fn_dest['folders'][data_folder], s)))
                #shutil.rmtree(os.path.join(fn_dest['folders'][data_folder], s))
            else:
                logger.info('Skipping')
                continue
        logger.info('Archiving {}'.format(s))
        #fstru, chk = archive_data(bird, sess, fold_type=data_folder, locations=locations, mode='xz', use_scratch=True)
        #bird_archived.append([s, fstru, chk])
            
    logger.info('Done making archive for {0} sessions of bird {1}'.format(len(bird_archived), bird))

archive_bird(bird, locations=locations, overwrite=False)

2017-02-02 23:57:57,337 root         INFO     2016-07-11_14-49-38_225 already present in /mnt/cube/earneodo/archive/bci_zf/raw_data/z017/
2017-02-02 23:57:57,337 root         WARNING  Will delete /mnt/cube/earneodo/bci_zf/raw_data/z017/2016-07-11_14-49-38_225 to overwrite
2017-02-02 23:57:57,338 root         INFO     Archiving /mnt/cube/earneodo/bci_zf/raw_data/z017/2016-07-11_14-49-38_225
2017-02-02 23:57:57,339 root         INFO     2016-06-29_09-00-36_225 already present in /mnt/cube/earneodo/archive/bci_zf/raw_data/z017/
2017-02-02 23:57:57,339 root         WARNING  Will delete /mnt/cube/earneodo/bci_zf/raw_data/z017/2016-06-29_09-00-36_225 to overwrite
2017-02-02 23:57:57,340 root         INFO     Archiving /mnt/cube/earneodo/bci_zf/raw_data/z017/2016-06-29_09-00-36_225
2017-02-02 23:57:57,340 root         INFO     2016-06-19_11-37-25_150 already present in /mnt/cube/earneodo/archive/bci_zf/raw_data/z017/
2017-02-02 23:57:57,341 root         WARNING  Will delete /mnt/cube/earneodo

In [40]:
tuvieja

NameError: name 'tuvieja' is not defined